https://github.com/coastalcph/lex-glue/blob/main/experiments/unfair_tos.py

In [ ]:
!pip install langchain
!pip install openai
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 8.5 MB/s eta 0:00:00


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import os
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain
from langchain.chains import SequentialChain
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.llms import OpenAI
import pandas as pd
import numpy as np
import random
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import re
from datasets import load_dataset

In [ ]:
RANDOM_SEED = 16

In [ ]:
from secret_key import openapi_key

In [ ]:


os.environ['OPENAI_API_KEY'] = openapi_key


In [ ]:

test_dataset = load_dataset("lex_glue", "unfair_tos", split="test")

val_dataset = load_dataset("lex_glue", "unfair_tos", split="validation")

Generating train split:   0%|          | 0/5532 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1607 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2275 [00:00<?, ? examples/s]

In [ ]:
print(val_dataset)

print(test_dataset)

Dataset({
    features: ['text', 'labels'],
    num_rows: 2275
})
Dataset({
    features: ['text', 'labels'],
    num_rows: 1607
})


In [ ]:
text_list_test = test_dataset["text"]
labels_list_test = test_dataset["labels"]

text_list_val = val_dataset["text"]
labels_list_val = val_dataset["labels"]

# Convert labels to a list of lists


In [ ]:
def fun(s):

    return len(s)

f = sorted(labels_list_val,key = fun)

In [ ]:
def convert(l):

    m = {0: "Arbitration", 1: "Choice of Law",
         2: "Content Removal" , 3: "Contract by Using",
         4: "Jurisdiction", 5: "Limitation of Liability",
         6: "Unilateral Change"  , 7: "Unilateral Termination"  }



    l1 = []
    s = " and "

    for i in l:

        l1.append(m[i])

    return s.join(l1)


def to_text(texts,labels):

    m_texts = []
    m_labels = []

    for l,t in zip(labels,texts):

        l_new = convert(l)
        m_labels.append(l_new)
        m_texts.append(t)

    return m_texts,m_labels



def process(texts,labels):

    m_texts = []
    m_labels = []

    for l,t in zip(labels,texts):

        if(len(l) >= 1):


            m_labels.append(l)
            m_texts.append(t)

    return m_texts,m_labels



def cus_sort(s):

    return s[1]




def process2(texts,labels):

    m_texts = []
    m_labels = []

    data = []

    for l,t in zip(labels,texts):

        if(len(l) >= 1):

            m_labels.append("Unfair")

        else:

            m_labels.append("Fair")

        m_texts.append(t)


    return m_texts, m_labels





def gen_data(X,Y):

    data = []


    for x,y in zip(X,Y):

        obj = {
            "agreements" : x,
            "categories" : y
        }

        data.append(obj)

    random.seed(64)
    random.shuffle(data)

    return data


In [ ]:
X_test_ , y_test_ = process2(text_list_test , labels_list_test)

test = gen_data(X_test_ , y_test_)





X_val_ , y_val_ = process2(text_list_val , labels_list_val)

val = gen_data(X_val_ , y_val_)

val_data = val[:50]

In [ ]:
y_test_.count("Unfair") , y_test_.count("Fair")

(172, 1435)

In [ ]:
def gen_few_shots(data,m= 2, n = 3):

    Unfair = 0
    Fair = 0

    data_ = []


    for x in data:

        if(x["categories"] == "Unfair" and Unfair <= m ):

            obj = {
                "agreements" : x["agreements"] ,
                "categories" : "Unfair"
            }

            data_.append(obj)

            Unfair += 1

        elif(x["categories"] == "Fair" and Fair <= n ):

            obj = {
                "agreements" : x["agreements"] ,
                "categories" : "Fair"
            }

            data_.append(obj)

            Fair += 1



    random.seed(42)
    random.shuffle(data_)


    return data_


few_shot_examples = gen_few_shots(val_data)


test_data = gen_few_shots(test, m = 20, n = 30)


In [ ]:
few_shot_examples

[{'agreements': 'releases , disclaimers , liability limits and indemnification \n',
  'categories': 'Fair'},
 {'agreements': 'you must not allow others to use your account credentials and you must safeguard the confidentiality of those credentials . \n',
  'categories': 'Fair'},
 {'agreements': "`` service '' means all features , applications , content and downloads offered by linden lab , including its websites , servers , software , linden content , and user content as those terms are defined herein . \n",
  'categories': 'Fair'},
 {'agreements': 'oculus reserves the right to change or modify these terms in the future at any time and at our sole discretion . \n',
  'categories': 'Unfair'},
 {'agreements': 'you can lose your user name and persona in the service as a result of account termination or limitation , as well as any benefits , privileges , earned items and purchased items associated with your use of the service , and supercell is under no obligation to compensate you for any

In [ ]:
few_shot_examples_cot = [
    {"agreements": "you must not allow others to use your account credentials and you must safeguard the confidentiality of those credentials . \n",
     "categories" : "The clause highlights the significance of maintaining the security of the account. The sentence reinforces the account holder's responsibility for their credentials. It establishes a clear expectation that the account holder should not share their login information with unauthorized individuals. The sentence is Fair",
   },

     {
      "agreements"  :  "releases , disclaimers , liability limits and indemnification . \n",
      "categories" : "Releases refer to provisions that outline specific situations or claims that one or both parties agree not to hold the other party liable for. Disclaimers are statements that clarify limitations on warranties, representations, or guarantees about the products or services being provided. Liability limits set boundaries on the financial responsibility of each party in case of damages, losses, or other liabilities arising from the business relationship. The sentence is Fair" ,


      },
    {   "agreements" : "6.2 your access to the products may be occasionally restricted to allow for repairs , maintenance or the introduction of new facilities or products . \n",
        "categories" : 'The sentence mentions that access "may be occasionally restricted," but it should clarify the frequency and duration of these restrictions. The agreement should clearly state that the service provider will provide reasonable advance notice of any planned restrictions. The sentence is Unfair',
    } ,

    {"agreements" : "oculus reserves the right to change or modify these terms in the future at any time and at our sole discretio. \n" ,
     "categories" : "Granting the service provider the sole discretion to change terms creates an imbalance of power in favor of the provider. The sentence allows the service provider to make changes without any prior notice or user consent. With such broad discretion, the service provider might be able to make changes that benefit them more than the user. The sentence is Unfair" , }  ,

    {"agreements": '11.1 the service is a united states-based service . \n',
     "categories" : " Clearly stating that the service is United States-based informs users about the geographic location of the service's operations and infrastructure.  Including this statement makes the terms of the service agreement clear and reduces the possibility of misunderstandings or disputes arising from differing assumptions about the service's location. The sentence is Fair" ,
     } ,


]

In [ ]:
few_shot_examples_cot

[{'agreements': 'you must not allow others to use your account credentials and you must safeguard the confidentiality of those credentials . \n',
  'categories': "The clause highlights the significance of maintaining the security of the account. The sentence reinforces the account holder's responsibility for their credentials. It establishes a clear expectation that the account holder should not share their login information with unauthorized individuals. The sentence is Fair"},
 {'agreements': 'releases , disclaimers , liability limits and indemnification . \n',
  'categories': 'Releases refer to provisions that outline specific situations or claims that one or both parties agree not to hold the other party liable for. Disclaimers are statements that clarify limitations on warranties, representations, or guarantees about the products or services being provided. Liability limits set boundaries on the financial responsibility of each party in case of damages, losses, or other liabilitie

In [ ]:
# prompt_template = '''
# Text: {text}
# {class}
# '''

# example_prompt = PromptTemplate(input_variables=["text","class"],template = prompt_template)

# print(example_prompt.format(**examples[0]))


prompt_template2 = '''
Agreements: {agreements}
Categories: {categories}
'''

example_prompt2 = PromptTemplate(input_variables=["agreements","categories"],template = prompt_template2)

print(example_prompt2.format(**few_shot_examples[0]))





example_prompt3 = PromptTemplate(input_variables=["agreements","categories"],template = prompt_template2)

print(example_prompt3.format(**few_shot_examples_cot[0]))







Agreements: releases , disclaimers , liability limits and indemnification 

Categories: Fair


Agreements: you must not allow others to use your account credentials and you must safeguard the confidentiality of those credentials . 

Categories: The clause highlights the significance of maintaining the security of the account. The sentence reinforces the account holder's responsibility for their credentials. It establishes a clear expectation that the account holder should not share their login information with unauthorized individuals. The sentence is Fair



# Prediction

# Using few shot learning with my default prompt templete

In [ ]:
def trim(s):

    s = s.replace("-", " ").replace("_", " ")

    pattern = r'(Fair|Unfair)'

    matches = re.findall(pattern, s)

    return matches[0].strip()



# def trim(s,flag = True):


#     s = s.replace("-", " ").replace("_", " ")

#     pattern = r'(Arbitration|Choice of Law|Content Removal|Contract by Using|Jurisdiction|Limitation of Liability|Unilateral Change|Unilateral Termination)'

#     matches = re.findall(pattern, s)

#     if flag == True:

#       return " ".join(matches)

#     else:

#       return set(matches)



# def pseudo_acc(pred_y,acc_y):

#   for i in pred_y:

#     if(i in acc_y):

#       return (True,True)

#   return (False,True)




def eval(extraction_chain, example,s = 40 , few_shot = True):

    preds = []
    actual = []

    for x in tqdm(example):

        time.sleep(s)

        if(few_shot):

            p = extraction_chain.predict(agreements = x["agreements"] )

        else:

            p = extraction_chain.run(x["agreements"] )

        p = trim(p)


        a = x["categories"]

        preds.append(p)
        actual.append(a)



    # print(preds)
    # print(actual)

    # print(pseudo_preds)
    # print(pseudo_actual)

    return preds,actual, accuracy_score(actual, preds)










In [ ]:
# pseudo_acc(pred_y,acc_y)

In [ ]:
# m = {0: "Arbitration", 1: "Choice of Law",
#          2: "Content Removal" , 3: "Contract by Using",
#          4: "Jurisdiction", 5: "Limitation of Liability",
#          6: "Unilateral Change"  , 7: "Unilateral Termination"  }

0.7628865979381443

In [ ]:
prefix = """You are a service agreement analyst. Classify the given agreements, which belongs to any of the eight categories. Please read each requirement carefully and determine its classification based on the definitions provided below.

                All Categories name are given below:

                <Categories>: "Arbitration", "Choice of Law", Content Removal", "Contract by Using", "Jurisdiction" , "Limitation of Liability", "Unilateral Change", "Unilateral Termination"

                Defination of all Categories:

                <Arbitration>: a legal process in which disputes between parties are resolved by an impartial third party (arbitrator) outside of the court system, with the decision typically being binding and enforceable.

                <Choice of Law>: refers to the decision made in a legal contract regarding which jurisdiction's laws will govern the interpretation and enforcement of the contract's terms and disputes arising from it.

                <Content Removal>: pertains to the process of eliminating or taking down specific material from a platform or website, often in response to legal requests or violations of terms of service.

                <Contract by Using>: signifies the formation of a legally binding agreement through actions or conduct, rather than explicit written or verbal communication.

                <Jurisdiction>: refers to the authority of a court or legal system to hear and make decisions on legal matters within a specific geographic area or over particular subjects.

                <Limitation of Liability>: is a provision in a legal document that restricts the extent to which parties can be held financially accountable for certain damages or losses.

                <Unilateral Change>: refers to a modification, adjustment, or alteration made by one party without requiring the consent or agreement of the other parties involved.

                <Unilateral Termination>: signifies the act of ending or canceling a contract or agreement by one party without the need for approval from the other parties.

                Please read each agreement statement and classify the category of the agreement. A single agreement can have multiple catergories
    """


prefix2 = """You are a service agreement analyst. Classify the given agreements, which belongs to any of the eight categories. Please read each requirement carefully and determine its classification based on the definitions provided below.

                All Categories name are given below:

                <Categories>: "Arbitration", "Choice of Law", Content Removal", "Contract by Using", "Jurisdiction" , "Limitation of Liability", "Unilateral Change", "Unilateral Termination"

                Defination of all Categories:

                <Arbitration>: a legal process in which disputes between parties are resolved by an impartial third party (arbitrator) outside of the court system, with the decision typically being binding and enforceable.

                <Choice of Law>: refers to the decision made in a legal contract regarding which jurisdiction's laws will govern the interpretation and enforcement of the contract's terms and disputes arising from it.

                <Content Removal>: pertains to the process of eliminating or taking down specific material from a platform or website, often in response to legal requests or violations of terms of service.

                <Contract by Using>: signifies the formation of a legally binding agreement through actions or conduct, rather than explicit written or verbal communication.

                <Jurisdiction>: refers to the authority of a court or legal system to hear and make decisions on legal matters within a specific geographic area or over particular subjects.

                <Limitation of Liability>: is a provision in a legal document that restricts the extent to which parties can be held financially accountable for certain damages or losses.

                <Unilateral Change>: refers to a modification, adjustment, or alteration made by one party without requiring the consent or agreement of the other parties involved.

                <Unilateral Termination>: signifies the act of ending or canceling a contract or agreement by one party without the need for approval from the other parties.

                Please read each agreement statement and classify the category of the agreement.
    """





prefix3 = """You are a service agreement analyst. Classify the given agreements as "Fair" or "UnFair" categories. Please read each agreement carefully and determine its classification based on the definitions provided below.

                <Fair>: Functional requirements specify what the software system should do. They define the specific functionalities, operations, and interactions the system needs to support. Functional requirements describe the system's behavior in response to different inputs and events.

                <Unfair>: Non-functional requirements, often referred to as quality attributes or constraints, specify how the system should perform its functions. These requirements describe characteristics that are critical to the overall user experience, such as performance, security, scalability, usability, and reliability.

                Please read each requirement statement and classify it as either "Fair" or "UnFair" based on the provided definitions.
    """



prefix_cot = """You are a service agreement analyst. Categoried the given agreements as "Fair" or "UnFair" based on the examples provided below.


"""


# final_prompt = FewShotPromptTemplate(
#     examples=few_shot_examples,
#     example_prompt=example_prompt2,
#     suffix= """Agreements: {agreements} \nCategories: """,
#     input_variables=["agreements"],
#     prefix= prefix
# )


# print(final_prompt.format(agreements = test_examples[3]["agreements"] ))




# final_prompt = FewShotPromptTemplate(
#     examples=few_shot_examples,
#     example_prompt=example_prompt2,
#     suffix= """Agreements: {agreements} \nCategories: """,
#     input_variables=["agreements"],
#     prefix= prefix
# )





final_prompt = FewShotPromptTemplate(
    examples= few_shot_examples,
    example_prompt=example_prompt2,
    suffix= """Agreements: {agreements} \nCategories: """,
    input_variables=["agreements"],
    prefix= prefix3
)



print(final_prompt.format(agreements = test_data[3]["agreements"] ))


print()
print("*"*100)
print()
print("*"*100)
print()



final_prompt2 = FewShotPromptTemplate(
    examples= few_shot_examples_cot,
    example_prompt= example_prompt3,
    suffix= """Agreements: {agreements} \nCategories: """,
    input_variables=["agreements"],
    prefix= prefix_cot
)



print(final_prompt2.format(agreements = test_data[3]["agreements"] ))




You are a service agreement analyst. Classify the given agreements as "Fair" or "UnFair" categories. Please read each agreement carefully and determine its classification based on the definitions provided below.

                <Fair>: Functional requirements specify what the software system should do. They define the specific functionalities, operations, and interactions the system needs to support. Functional requirements describe the system's behavior in response to different inputs and events.

                <Unfair>: Non-functional requirements, often referred to as quality attributes or constraints, specify how the system should perform its functions. These requirements describe characteristics that are critical to the overall user experience, such as performance, security, scalability, usability, and reliability.

                Please read each requirement statement and classify it as either "Fair" or "UnFair" based on the provided definitions.
    


Agreements: releases ,

# FeW shot

In [ ]:
llm = OpenAI(temperature=0.2)

In [ ]:
extraction_chain = LLMChain(llm = llm , prompt = final_prompt , output_key= "categories" )

In [ ]:
preds,actual, accuracy = eval(extraction_chain,test_data)

100%|██████████| 52/52 [35:01<00:00, 40.42s/it]


In [ ]:
accuracy

0.6923076923076923

# Prediction

# Using Zero shot

In [ ]:
llm = OpenAI(temperature=0.2)

In [ ]:
template = """  You are a service agreement analyst. Classify the given agreements as "Fair" or "UnFair" categories. Please read each agreement carefully and determine its classification based on the definitions provided below.

               <Fair>: Functional requirements specify what the software system should do. They define the specific functionalities, operations, and interactions the system needs to support. Functional requirements describe the system's behavior in response to different inputs and events.

                <Unfair>: Non-functional requirements, often referred to as quality attributes or constraints, specify how the system should perform its functions. These requirements describe characteristics that are critical to the overall user experience, such as performance, security, scalability, usability, and reliability.

                Please read each requirement statement, which is delimited by triple backticks and classify it as either "Fair" or "UnFair" based on the provided definitions.

                ```
                {agreements}
                ```
    """



prompt_template_name = PromptTemplate(
    input_variables =['agreements'],
    template = template
)



p = prompt_template_name.format(agreements = test_data[3]["agreements"] )
print(p)

  You are a service agreement analyst. Classify the given agreements as "Fair" or "UnFair" categories. Please read each agreement carefully and determine its classification based on the definitions provided below.

               <Fair>: Functional requirements specify what the software system should do. They define the specific functionalities, operations, and interactions the system needs to support. Functional requirements describe the system's behavior in response to different inputs and events.

                <Unfair>: Non-functional requirements, often referred to as quality attributes or constraints, specify how the system should perform its functions. These requirements describe characteristics that are critical to the overall user experience, such as performance, security, scalability, usability, and reliability.

                Please read each requirement statement, which is delimited by triple backticks and classify it as either "Fair" or "UnFair" based on the provided d

In [ ]:
extraction_chain = LLMChain(llm=llm, prompt=prompt_template_name)

preds,actual, accuracy = eval(extraction_chain,test_data,few_shot = False)

100%|██████████| 52/52 [34:59<00:00, 40.38s/it]


In [ ]:
preds,actual

In [ ]:
accuracy

0.46153846153846156

# Chain of thoughts

In [ ]:
llm = OpenAI(temperature=0.2)
extraction_chain = LLMChain(llm = llm , prompt = final_prompt2 , output_key= "categories" )
preds,actual, accuracy = eval(extraction_chain,test_data)

100%|██████████| 52/52 [35:45<00:00, 41.25s/it]


In [ ]:
accuracy

0.6730769230769231

# visualize few examples